In [ ]:
bucket = 'de-akira'
prefix = 'sagemaker/unsupervised_category'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
import numpy as np
from generate_example_data import generate_griffiths_data, plot_topic_data
import io
import os
import time
import json
import sys
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import scipy
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker.predictor import csv_serializer, json_deserializer

import neologdn
import os
import urllib.request
from matplotlib.font_manager import FontProperties
import MeCab
from __future__ import unicode_literals
import re
import unicodedata
import sagemaker
from sagemaker import get_execution_role
import boto3
import json
import pandas as pd
import warnings
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import codecs
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#chaneg dummies to matrix
from scipy.sparse import csr_matrix,coo_matrix

import multiprocessing

from multiprocessing import Pool

import pickle

import math
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import  roc_auc_score,f1_score

from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate,Conv1D, MaxPool1D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau,Callback
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
import os
import tarfile
import string
import numpy as np

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas(desc="hoge progress: ")

import time
import gensim
tqdm.pandas()

import numpy as np
import math

from sudachipy import tokenizer
from sudachipy import dictionary


tokenizer_obj = dictionary.Dictionary().create()

In [ ]:
mode = tokenizer.Tokenizer.SplitMode.A
def wakati_by_sudachi(txt):
    processed=[m.normalized_form() for m in tokenizer_obj.tokenize(txt, mode)]
    return " ".join(processed)

In [ ]:
#tokenize japanase words
def wakati_by_mecab(text):
    tagger = MeCab.Tagger('-Ochasen -d /usr/lib64/mecab/dic/mecab-ipadic-neologd')
    tagger.parse('') 
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        pos = node.feature.split(",")[0]
        #if pos in ["名詞","動詞","形容詞","記号"]:   # 対象とする品詞
        if pos in ["名詞"]:   # 対象とする品詞
            word = node.surface
            word_list.append(word)
        node = node.next
    return " ".join(word_list)

In [ ]:
def remove_chars(row):
    cust_list=['\d+','■','\n','#','◇','①','②','③','④','【】','】','【','☆','_','%','「','」','★','/']
    del_list = string.ascii_letters+'"#$%&\'()*+,-./:;<=>@[\\]^_`{|}~'
    for i in del_list:
        row = row.str.replace(i,'')
    for i in cust_list :
        row = row.str.replace(i,'')
    return row

In [ ]:
def clean_contractions(text):
    mapping ={"矢張り":"やはり","迚も":"とても","迚も":"とても","此れ":"これ","其れ":"それ","此の":"この","此の":"この","可成":"かなり","兎に角":"とにかく","態々":"わざわざ"
             ,"です":"","だ":"","ます":"","て":"","た":""}
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [ ]:
df_mutilabel=pd.read_excel('/home/ec2-user/SageMaker/Notebooks_For_CX_Usecases/Unsupervised_Category/raw_data/gr_comment.xlsx')

In [ ]:
df_mutilabel['comment'] = remove_chars(df_mutilabel['comment'])
df_mutilabel['comment'].replace('', np.nan, inplace=True)
df_mutilabel['comment']=df_mutilabel['comment'].apply(neologdn.normalize)
pool = Pool(processes=multiprocessing.cpu_count())
df_mutilabel['comment'] = pool.map(wakati_by_mecab,df_mutilabel['comment'])
pool.close() 
pool.join()

pool = Pool(processes=multiprocessing.cpu_count())
df_mutilabel['comment'] = pool.map(clean_contractions,df_mutilabel['comment'])
pool.close() 
pool.join()
df_mutilabel['comment'].replace('', np.nan, inplace=True)

def count_len(s):
    return len(s)
df_mutilabel['comment'].apply(count_len).sort_values(ascending=False)

In [ ]:
# define nums of docs, voc size etc.
num_documents = 1815
num_topics = 10
vocabulary_size = 50

In [ ]:
df_mutilabel["comment"].fillna("_##_",inplace=True)

In [ ]:
train_X = df_mutilabel["comment"].values
tokenizer = Tokenizer(num_words=600)
tokenizer.fit_on_texts(list(df_mutilabel['comment']))
train_X = tokenizer.texts_to_sequences(train_X)
train_X = pad_sequences(train_X, padding="post",maxlen=vocabulary_size)

In [ ]:
train_X.shape

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# creating matrix
input_matrix = train_X
print('Input shape: ', input_matrix.shape)
# splitting into two matrices of second matrix by size
second_size = 300/1815

train_x, val_x = train_test_split(input_matrix, test_size=second_size)

print('X1 shape: ', train_x.shape)
print('X2 shape: ', val_x.shape)

data_training = (train_x, np.zeros(int(len(train_x))))
data_validation=(val_x, np.zeros(int(len(val_x))))

In [ ]:
data_training = (train_X, np.zeros(int(len(train_X))))

In [ ]:
print('First training document = {}'.format(train_X[0]))
print('\nVocabulary size = {}'.format(vocabulary_size))

In [ ]:
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, data_training[0].astype('float32'))
buf.seek(0)

key = 'ntm.data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)

In [ ]:
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, data_validation[0].astype('float32'))
buf.seek(0)

key = 'ntm.data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation', key)).upload_fileobj(buf)
s3_train_data = 's3://{}/{}/validation/{}'.format(bucket, prefix, key)

In [ ]:
s3_train_data

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'ntm')

In [ ]:
sess = sagemaker.Session()

ntm = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.c4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
ntm.set_hyperparameters(num_topics=num_topics,
                        feature_dim=vocabulary_size)

ntm.fit({'train': s3_train_data})

In [ ]:
ntm_predictor = ntm.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

#real time to get predictor from endpoint
ntm_predictor=sagemaker.predictor.RealTimePredictor(endpoint="ntm-endpoint")

In [ ]:
ntm_predictor.content_type = 'text/csv'
ntm_predictor.serializer = csv_serializer
ntm_predictor.deserializer = json_deserializer

In [ ]:
results = ntm_predictor.predict(train_X)

In [ ]:
predictions = np.array([prediction['topic_weights'] for prediction in results['predictions']])

In [ ]:
def get_category(listOfStr):
    value=max(listOfStr)
    dictOfWords = { i : listOfStr[i] for i in range(0, len(listOfStr) ) }
    for index, num in dictOfWords.items():
        if num == value:
            return index
        

In [ ]:
cate_list=[]
for lst in range(len(predictions)):
    cate_list.append(get_category(predictions[lst]))

In [ ]:
len(cate_list)

In [ ]:
s=pd.Series(cate_list) 

In [ ]:
data={"comment":df_mutilabel['comment'].tolist(),"category":s.tolist()}

In [ ]:
df = pd.DataFrame(data) 

In [ ]:
df[(df['category']==7)]

In [ ]:
df_mutilabel=df_mutilabel['comment']

In [ ]:
df_mutilabel['category']=pd.Series(cate_list) 